In [ ]:
import pandas as pd
from tqdm import tqdm
import torch
import torchaudio
from torch.utils.data import Dataset, DataLoader

from utils.task import load_checkpoint
from utils.hparams import get_hparams_from_file
from model.models import SynthesizerTrn
from text.symbols import symbols
from text import tokenizer


def get_text(text: str, hps) -> torch.LongTensor:
    text_norm = tokenizer(text, hps.data.text_cleaners, language=hps.data.language)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [ ]:
dataset_path = "filelists/madasr23_test.csv"
output_path = "/path/to/output/directory"
data = pd.read_csv(dataset_path, sep="|")
print(data.head())

## MADASR23 batch inference


In [ ]:
model = "custom_base"
hps = get_hparams_from_file(f"./datasets/{model}/config.yaml")

In [ ]:
net_g = SynthesizerTrn(len(symbols), hps.data.n_mels if hps.data.use_mel else hps.data.n_fft // 2 + 1, hps.train.segment_size // hps.data.hop_length, n_speakers=hps.data.n_speakers, **hps.model).cuda()
_ = net_g.eval()

_ = load_checkpoint(f"./datasets/{model}/logs/G_15000.pth", net_g, None)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, dataframe, hps):
        self.data = dataframe
        self.hps = hps

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sid_idx = self.data["sid_idx"][idx]
        sid = self.data["sid"][idx]
        phonemes = self.data["phonemes"][idx]
        stn_tst = get_text(phonemes, self.hps)
        return sid_idx, sid, stn_tst, idx


# Initialize the dataset and data loader
dataset = MyDataset(data, hps)
data_loader = DataLoader(dataset, batch_size=1, num_workers=8)

for sid_idx, spk_id, stn_tst, i in tqdm(data_loader):
    sid_idx = int(sid_idx)
    spk_id = int(spk_id)
    i = int(i)
    stn_tst = stn_tst[0]
    with torch.no_grad():
        x_tst = stn_tst.cuda().unsqueeze(0)
        x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cuda()
        sid = torch.LongTensor([sid_idx]).cuda()
        audio = net_g.infer(x_tst, x_tst_lengths, sid=sid, noise_scale=0.667,
                            noise_scale_w=0.8, length_scale=1)[0][0].data.cpu()
        torchaudio.save(f"{output_path}/{spk_id}_{i}.wav", audio,
                        hps.data.sample_rate, bits_per_sample=hps.data.bits_per_sample)

print("Done!")

### Voice Conversion

TODO: Add batch inference for voice conversion
